In [ ]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
#     max_tokens=None,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [6]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://job-boards.greenhouse.io/coursera/jobs/5554771004?gh_src=6d6e4f994us")
page_data=loader.load().pop().page_content
page_data

"Job Application for Software Engineer I, Internal Tools (Frontend) at CourseraNewSoftware Engineer I, Internal Tools (Frontend)IndiaApplyCoursera was launched in 2012 by Andrew Ng and Daphne Koller, with a mission to provide universal access to world-class learning. It is now one of the largest online learning platforms in the world, with 175 million registered learners as of March 31, 2025. Coursera partners with over 350 leading universities and industry leaders to offer a broad catalog of content and credentials, including courses, Specializations, Professional Certificates, and degrees. Coursera’s platform innovations enable instructors to deliver scalable, personalized, and verified learning experiences to their learners. Institutions worldwide rely on Coursera to upskill and reskill their employees, citizens, and students in high-demand fields such as GenAI, data science, technology, and business. Coursera is a Delaware public benefit corporation and a B Corp.\nJoin us in our mi

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
       """
### SCRAPED TEXT FROM WEBSITE
{page_data}

### INSTRUCTION
The scraped text is from a careers page.
Extract all job postings and return them **only** in **valid JSON format** using the following keys: `role`, `experience`, `skills`, and `description`.

Do **not** include any explanation, heading, or preamble.  
Do **not** wrap the JSON in triple backticks.  
Do **not** say anything like "Here's the result" or "Below is the JSON".

Just return **pure JSON**, beginning with `{{` and ending with `}}`.

### OUTPUT:
"""

)

chain_extract= prompt_extract | llm
res=chain_extract.invoke(input={'page_data': page_data})
print(res.content)
print(type(res.content))


{
"role": "Software Engineer I, Internal Tools (Frontend)",
"experience": "1-2 years",
"skills": ["Strong foundation in programming concepts, data structures, and algorithms", "Familiarity with web development technologies (HTML, CSS, JavaScript)", "Exposure to front-end frameworks like React, Angular, or Vue.js is a plus", "Basic understanding of REST APIs or CRUD operations", "Good communication skills and a proactive mindset"],
"description": "Assist in building and improving web-based internal tools used by various teams at Coursera. Write clean, efficient, and maintainable code with support from your team. Work with product managers and other engineers to understand user needs and deliver solutions. Learn and apply best practices for full-stack development, including front-end and back-end systems. Help maintain documentation, fix bugs, and test features to ensure quality. Contribute to team discussions and share your ideas during design and planning phases."
}
<class 'str'>


In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res
print(type(json_res))

<class 'dict'>


In [12]:
import pandas as pd

df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [20]:
import chromadb
import uuid
client = chromadb.PersistentClient()
client.delete_collection(name="portfolio")

collection= client.create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])


In [22]:
links=collection.query(query_texts=["Experience in Python","Experience in React"],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [24]:
job= json_res
job['skills']

['Strong foundation in programming concepts, data structures, and algorithms',
 'Familiarity with web development technologies (HTML, CSS, JavaScript)',
 'Exposure to front-end frameworks like React, Angular, or Vue.js is a plus',
 'Basic understanding of REST APIs or CRUD operations',
 'Good communication skills and a proactive mindset']

In [26]:
job

{'role': 'Software Engineer I, Internal Tools (Frontend)',
 'experience': '1-2 years',
 'skills': ['Strong foundation in programming concepts, data structures, and algorithms',
  'Familiarity with web development technologies (HTML, CSS, JavaScript)',
  'Exposure to front-end frameworks like React, Angular, or Vue.js is a plus',
  'Basic understanding of REST APIs or CRUD operations',
  'Good communication skills and a proactive mindset'],
 'description': 'Assist in building and improving web-based internal tools used by various teams at Coursera. Write clean, efficient, and maintainable code with support from your team. Work with product managers and other engineers to understand user needs and deliver solutions. Learn and apply best practices for full-stack development, including front-end and back-end systems. Help maintain documentation, fix bugs, and test features to ensure quality. Contribute to team discussions and share your ideas during design and planning phases.'}

In [27]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Ekta Goel, a final-year Integrated M.Tech student specializing in Computer Science and Engineering, currently interning at Intel Corporation as a Data Engineering and MLOps Intern. 

        You are writing a cold email expressing interest in the role described above. Highlight your hands-on experience in:

        - Building enterprise-grade CI/CD MLOps pipelines using Streamlit, GitHub Actions, and Databricks CLI.
        - Automating data quality checks with Snowflake and Python (1M+ records), reducing manual work by 70%.
        - Developing LLM-based data extraction pipelines using LangChain and Trafilatura.
        - Constructing end-to-end ML/NLP pipelines using Whisper ASR and BERT, achieving 95.4% accuracy.
        - Creating scalable ingestion and transformation pipelines with Azure Data Factory, Snowflake, and Databricks.
        - Engineering real-time monitoring tools and approval portals using PowerApps, MongoDB, Flask, and REST APIs.

        Also include links to relevant projects and portfolios that showcase your capability:
        - GitHub Webhook Monitoring System: https://github.com/ektagoel-12/webhook-repo  
        - Audio-to-Text Punctuation Prediction: https://github.com/ektagoel-12/Python-Projects/tree/main/Audio_to_test_punctuation_prediction  
        - Personal Portfolio: https://ektagoel.site/

        Write a concise, professional cold email tailored to the job description. You are not representing any company—just showcasing your individual capabilities and interest in the position.

        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer I, Internal Tools (Frontend) Role

Dear Hiring Manager,

I'm excited to express my interest in the Software Engineer I, Internal Tools (Frontend) position at Coursera. With a strong foundation in programming concepts, data structures, and algorithms, I'm confident in my ability to contribute to building and improving web-based internal tools.

As a final-year Integrated M.Tech student in Computer Science and Engineering, I've had the opportunity to develop a range of skills relevant to this role. My experience as a Data Engineering and MLOps Intern at Intel Corporation has given me hands-on experience in building enterprise-grade CI/CD MLOps pipelines, automating data quality checks, and developing scalable ingestion and transformation pipelines.

In particular, I've worked on projects that demonstrate my proficiency in frontend development, including:

* Building a GitHub Webhook Monitoring System (https://github.com/ektagoel-12/webhook-repo)